In [6]:
import win32com.client
import os
word=win32com.client.Dispatch("word.Application")
word.visible=0

doc_pdf ="docs\Original.pdf"
input_file=os.path.abspath(doc_pdf)
wb = word.Documents.Open(input_file)
output_file=os.path.abspath(doc_pdf[0:-4] + "docx".format())
wb.SaveAs2(output_file,FileFormat=16)
print("pdf to doc")

In [11]:
import chardet
pdf_path = "docs\Original.pdf"

def detect_file_encoding(pdf_path):
    with open(pdf_path, 'rb') as f:
        result = chardet.detect(f.read())
        return result['encoding']

    


In [14]:
detect_file_encoding("docs\Original.pdf")

In [1]:
import win32com.client
word = win32com.client.Dispatch("Word.Application")
print("Word application launched successfully!")

com_error: (-2147221005, 'Invalid class string', None, None)